In [1]:
import sys
sys.path.append('/Users/davison/Software/almeida-semantica')
from src.ConectorBD import *
import cltk as cltk
import pandas as pd
from pandasql import sqldf
import os

In [2]:
import koinenlp

In [3]:
lxx = '/Users/davison/Software/almeida-semantica/db/lxx.db'

In [4]:
lxx_df = pd.read_sql_query("""
                           SELECT 
                                src_name
                                , book_name
                                , chapter_nr
                                , verse_nr
                                , group_concat(word, ' ') as verse_text
                           FROM 
                                content
                           group by 
                                src_name
                                , book_name
                                , chapter_nr
                                , verse_nr
                           """
                           , ConectorBD(lxx).con)

In [33]:
lxx_db = pd.read_sql_query("select * from content", ConectorBD(lxx).con)
lxx_db.query('book_name == "Gen"').word_root

160235          ἐν
160236        ἀρχή
160237       ποιέω
160238           ὁ
160239        θεός
            ...   
594409          ἐν
594410           ὁ
594411       σορός
594412          ἐν
594413    Αἴγυπτος
Name: word_root, Length: 32566, dtype: object

In [6]:
strongs_df = pd.read_sql_query("""
                                SELECT
                                    *   
                                FROM
                                    strongs
                                """, ConectorBD('/Users/davison/Software/almeida-semantica/db/strongs.sqlite').con)

In [17]:
strongs_df.head()

,number,lemma,xlit,pronounce,description
0,H1,אָב,ʼâb,awb,"a primitive word; father, in a literal and imm..."
1,H2,אַב,ʼab,ab,(Aramaic) corresponding to אָב; {father}; father.
2,H3,אֵב,ʼêb,abe,from the same as אָבִיב; a green plant; greenn...
3,H4,אֵב,ʼêb,abe,(Aramaic) corresponding to אֵב; {a green plant...
4,H5,אֲבַגְתָא,ʼĂbagthâʼ,ab-ag-thaw',"of foreign origin; Abagtha, a eunuch of Xerxes..."


In [38]:
strongs_df['word_root'] = [koinenlp.normalize(i) for i in strongs_df.lemma]

In [39]:
strongs_df.word_root

0              אב
1              אב
2              אב
3              אב
4           אבגתא
           ...   
14293        ωστε
14294       ωτιον
14295     ωφελεια
14296      ωφελεω
14297    ωφελιμοσ
Name: word_root, Length: 14298, dtype: object

In [ ]:
palavra_lxx_por_indice = koinenlp.normalize(lxx_db.query('book_name == "Gen"').word_root.values[3])
wd_gnt_by_vs = 
# palavra_lxx_por_indice = lxx_db.query('book_name == "Gen"').word_root.values[1]
print(palavra_lxx_por_indice)
strong_por_lxx_word_root = strongs_df.query('word_root == "{}"'.format(palavra_lxx_por_indice))

strong_por_lxx_word_root

ο


,number,lemma,xlit,pronounce,description,word_root
12261,G3588,ὁ,ho,to,the definite article; the (sometimes to be sup...,ο


In [6]:
lxx_df.head()

,src_name,book_name,chapter_nr,verse_nr,verse_text
0,01.Gen.1.mlxx,Gen,1,1,ἐν ἀρχῇ ἐποίησεν ὁ θεὸς τὸν οὐρανὸν καὶ τὴν γῆν
1,01.Gen.1.mlxx,Gen,1,2,ἡ δὲ γῆ ἦν ἀόρατος καὶ ἀκατασκεύαστος καὶ σκότ...
2,01.Gen.1.mlxx,Gen,1,3,καὶ εἶπεν ὁ θεός γενηθήτω φῶς καὶ ἐγένετο φῶς
3,01.Gen.1.mlxx,Gen,1,4,καὶ εἶδεν ὁ θεὸς τὸ φῶς ὅτι καλόν καὶ διεχώρισ...
4,01.Gen.1.mlxx,Gen,1,5,καὶ ἐκάλεσεν ὁ θεὸς τὸ φῶς ἡμέραν καὶ τὸ σκότο...


In [7]:
sql_lxx_strongs = """
                    SELECT
                        l.src_name
                        , l.book_name
                        , l.chapter_nr
                        , l.verse_nr
                        , l.word
                        , l.word_root
                        , s.number
                        --, s.strongs_def
                    FROM
                        lxx_db l
                    LEFT JOIN    
                        strongs_df s on l.word_root = s.lemma
                    WHERE
                        l.word like '%ἐπιδέκατα%'
                        --s.number like '%1183'
                    """

In [8]:
lxx_strongs_df = sqldf(sql_lxx_strongs)

In [9]:
len(lxx_strongs_df)

9

In [58]:
lxx_strongs_df.head()

,src_name,book_name,chapter_nr,verse_nr,word,word_root,number
0,05.Num.mlxx,Num,18,24,ἐπιδέκατα,ἐπιδέκατος,None
1,40.Amos.mlxx,Amos,4,4,ἐπιδέκατα,ἐπιδέκατος,None
2,49.Malachi.mlxx,Mal,3,8,ἐπιδέκατα,ἐπιδέκατος,None
3,17.2Chron.mlxx,2Chr,31,5,ἐπιδέκατα,ἐπιδέκατος,None
4,17.2Chron.mlxx,2Chr,31,6,ἐπιδέκατα,ἐπιδέκατος,None


In [10]:
lxx_df.head()

,src_name,book_name,chapter_nr,verse_nr,verse_text
0,01.Gen.1.mlxx,Gen,1,1,ἐν ἀρχῇ ἐποίησεν ὁ θεὸς τὸν οὐρανὸν καὶ τὴν γῆν
1,01.Gen.1.mlxx,Gen,1,2,ἡ δὲ γῆ ἦν ἀόρατος καὶ ἀκατασκεύαστος καὶ σκότ...
2,01.Gen.1.mlxx,Gen,1,3,καὶ εἶπεν ὁ θεός γενηθήτω φῶς καὶ ἐγένετο φῶς
3,01.Gen.1.mlxx,Gen,1,4,καὶ εἶδεν ὁ θεὸς τὸ φῶς ὅτι καλόν καὶ διεχώρισ...
4,01.Gen.1.mlxx,Gen,1,5,καὶ ἐκάλεσεν ὁ θεὸς τὸ φῶς ἡμέραν καὶ τὸ σκότο...


In [11]:
gnt = '/Users/davison/Software/almeida-semantica/db/sbl.db'

In [12]:
gnt_df = pd.read_sql_query("SELECT * FROM words", ConectorBD(gnt).con)

In [13]:
gnt_grouped_df = pd.read_sql_query("""
                           SELECT
                                nr_sq_livro
                                , chapter
                                , verse
                                , group_concat(word, ' ') as words
                                , group_concat(lemma, ' ') as lemmas
                                , group_concat(strongs, ' ') as strongs
                            FROM 
                                words
                            group by
                                nr_sq_livro
                                , chapter
                                , verse""", ConectorBD(gnt).con)

In [17]:
gnt_df.head()

,book,chapter,verse,pos,morph,lemma,strongs,word,punct,word_nr,nr_sq_livro
0,Matt,1,1,N-,----NSF-,βίβλος,00976,Βίβλος,,1,40
1,Matt,1,1,N-,----GSF-,γένεσις,01078,γενέσεως,,2,40
2,Matt,1,1,N-,----GSM-,Ἰησοῦς,02424,Ἰησοῦ,,3,40
3,Matt,1,1,N-,----GSM-,Χριστός,05547,χριστοῦ,,4,40
4,Matt,1,1,N-,----GSM-,υἱός,05207,υἱοῦ,,5,40


In [43]:
gnt_df.loc[(gnt_df['nr_sq_livro'] == 45) & (gnt_df['chapter'] == 10) & (gnt_df['verse'] == 9)]

,book,chapter,verse,pos,morph,lemma,strongs,word,punct,word_nr,nr_sq_livro
87305,Rom,10,9,C-,--------,ὅτι,03754,ὅτι,,1,45
87306,Rom,10,9,C-,--------,ἐάν,01437,ἐὰν,⸀,2,45
87307,Rom,10,9,V-,2AAS-S--,ὁμολογέω,03670,ὁμολογήσῃς,,3,45
87308,Rom,10,9,P-,--------,ἐν,01722,ἐν,,4,45
87309,Rom,10,9,RA,----DSN-,ὁ,03588,τῷ,,5,45
87310,Rom,10,9,N-,----DSN-,στόμα,04750,στόματί,⸀,6,45
87311,Rom,10,9,RP,----GS--,σύ,04771,σου,,7,45
87312,Rom,10,9,N-,----ASM-,κύριος,02962,κύριον,,8,45
87313,Rom,10,9,N-,----ASM-,Ἰησοῦς,02424,Ἰησοῦν,",",9,45
87314,Rom,10,9,C-,--------,καί,02532,καὶ,,10,45


In [ ]:
gnt_grouped_df.values[0]

40

In [42]:
# palavra_grega = koinenlp.normalize(strongs_grk_df['lemma'][1])
import re
palavra_grega = koinenlp.normalize(gnt_df.loc[(gnt_df['nr_sq_livro'] == 45) & (gnt_df['chapter'] == 10) & (gnt_df['verse'] == 9)].values[2][5])
print(palavra_grega)
df_query = lxx_df["verse_text"].str.contains(palavra_grega, flags=re.IGNORECASE, regex=True)
df_query.value_counts()


ομολογεω


verse_text
False    30607
Name: count, dtype: int64

In [21]:
gnt_grouped_df.loc[(gnt_grouped_df['nr_sq_livro'] == 40) & (gnt_grouped_df['chapter'] == 19) & (gnt_grouped_df['verse'] == 16)]

,nr_sq_livro,chapter,verse,words,lemmas,strongs
631,40,19,16,Καὶ ἰδοὺ εἷς προσελθὼν αὐτῷ εἶπεν Διδάσκαλε τί...,καί ἰδού εἷς προσέρχομαι αὐτός λέγω διδάσκαλος...,02532 02400 01520 04334 00846 03004 01320 0510...


In [22]:
sql_lxx_expr ='''
                SELECT 
                    lxx.book_name
                    , lxx.chapter_nr
                    , lxx.verse_nr
                FROM 
                    lxx_df lxx
                WHERE 
                    lxx.verse_text like '%ἀγαπῶμεν%' and lxx.verse_text like '%ἐντολὰς%'
                '''

In [23]:
sql_lxx_strongs_expr ='''
                        SELECT 
                            lxx.book_name
                            , lxx.chapter_nr
                            , lxx.verse_nr
                        FROM 
                            lxx_df lxx
                        WHERE 
                            lxx.verse_text like '%ἀγαπῶμεν%' and lxx.verse_text like '%ἐντολὰς%'
                        '''

In [24]:
occs_lxx_expr = sqldf(sql_lxx_expr)

In [25]:
len(occs_lxx_expr)

0

In [26]:
occs_lxx_expr.head()

,book_name,chapter_nr,verse_nr


In [ ]:
# strongs_df_def = pd.read_sql_query("""SELECT * FROM 